In [1]:
import serial
import time

# --- Configuration ---
# IMPORTANT: Replace 'COM7' with the actual COM port of your SAM D21 board.
# You can find this in the Windows Device Manager.
SERIAL_PORT = 'COM7'
BAUD_RATE = 9600

def set_switches(ser, k1_state_binary, k2_state_binary):
    """
    Sends the binary string commands to the SAM D21 board.

    Args:
        ser: The pySerial serial object.
        k1_state_binary (str): An 8-character string of '1's and '0's for K1.
        k2_state_binary (str): An 8-character string of '1's and '0's for K2.
    """
    if len(k1_state_binary) != 8 or len(k2_state_binary) != 8:
        print("Error: Both binary strings must be exactly 8 characters long.")
        return

    print(f"Sending command: K1 = {k1_state_binary}, K2 = {k2_state_binary}")

    # The C code expects the K1 string first, then the K2 string.
    # We encode the strings to bytes before sending.
    ser.write(k1_state_binary.encode('ascii'))
    ser.write(k2_state_binary.encode('ascii'))

    # Wait a moment for the device to process and respond
    time.sleep(0.1)

    # Read and print the confirmation message from the device
    response = ser.readline().decode('ascii').strip()
    print(f"Device responded: {response}")

def main():
    """Main function to connect and send commands."""
    try:
        # Connect to the serial port
        with serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1) as ser:
            print(f"Successfully connected to {SERIAL_PORT} at {BAUD_RATE} baud.")
            
            # Wait for the initial message from the board
            time.sleep(2) # Give the board time to boot up
            welcome_message = ser.readline().decode('ascii').strip()
            print(f"Device says: {welcome_message}")

            # --- Example Commands ---
            # You can change these binary strings to control the outputs
            
            print("\n--- Test 1: Turning on K1 outputs 0-3 and K2 outputs 4-7 ---")
            set_switches(ser, "11110000", "00001111")
            time.sleep(2)

            print("\n--- Test 2: Turning all outputs ON ---")
            set_switches(ser, "11111111", "11111111")
            time.sleep(2)

            print("\n--- Test 3: Turning all outputs OFF ---")
            set_switches(ser, "00000000", "00000000")
            time.sleep(2)
            
            print("\nScript finished.")

    except serial.SerialException as e:
        print(f"Error: Could not open serial port {SERIAL_PORT}. Please check the port name and ensure it's not in use.")
        print(e)

if __name__ == "__main__":
    main()


Successfully connected to COM7 at 9600 baud.
Device says: 

--- Test 1: Turning on K1 outputs 0-3 and K2 outputs 4-7 ---
Sending command: K1 = 11110000, K2 = 00001111
Device responded: OK: K1=0xF0, K2=0x0F

--- Test 2: Turning all outputs ON ---
Sending command: K1 = 11111111, K2 = 11111111
Device responded: OK: K1=0xFF, K2=0xFF

--- Test 3: Turning all outputs OFF ---
Sending command: K1 = 00000000, K2 = 00000000
Device responded: OK: K1=0x00, K2=0x00

Script finished.
